# Homework 1: Monty Hall 


# Import 


In [1]:
using Random #for the "Shuffle!" function 
using StatsBase # for the "sample" function 


# Settings 

In [2]:
number_doors = 3 
num_simulation = 1000; 


# mutable structure 


create a mutable structure Door to represent a door with 2 attributes: 
    #whether or not it has been opened by the game show host 
    #what is behinf the door (goat or car)
    #it needs to be mutable so that the game show host can open it 

In [3]:
behind_the_door = Set{Symbol}([:goat, :car])  #setting up what will be behind the door

mutable struct Door 
    open::Bool  #dictates whether door is open or closed. Open = true  False = True 
    contains::Symbol  #this is what is behind the door (car or goat)
    
  
end 
    

# Functions

# pick_a_door_at_random()

returns an interger (1, 2, 3) at random, representing the door that the contestant chose.

In [4]:

function pick_a_door_at_random()  
    door_id_pick = (rand(1:number_doors)) #randomly picks a value 1-3 to signify which door the contestant has chosen
    return door_id_pick
end

door_id_pick = pick_a_door_at_random()

2

# set_up_door()

set up a function that constructs and returns an array of three Doors(Array{Door}) such that 
    #two doors have a goat behind them 
    #one door has a car behind it 
    #which door has the car is chosen at uniform random 
    #all doors are unopened 

In [5]:

function set_up_doors() #sets up the doors(2 goat, 1 car) all doors should be closed 
    doors = [Door(false, :goat) for _ in 1:number_doors]
    doors[pick_a_door_at_random()].contains = :car #uses the pick_a_door_at_random() function to put a car behind one of the doors 
    return doors 
end

doors = set_up_doors()

3-element Array{Door,1}:
 Door(false, :car) 
 Door(false, :goat)
 Door(false, :goat)

# open_a_door(doors::Array{Door}, first_door_id_pick::Int)


    #the door that the contestant picked is not opened 
    #the door with the car is not opened 
    #if two doors are viable to open, pick one at uniform random 

In [6]:

function open_a_door!(doors::Array{Door}, first_door_id_pick::Int64)
    #this determines which door the host should open (not the contestants not the one with a car)
    valid_doors = Array(1:number_doors)
    deleteat!(valid_doors, valid_doors .== first_door_id_pick) #taking the door the contestant choose out 
    deleteat!(valid_doors,  [doors[i].contains == :car for i in valid_doors]) #taking the door with the car out (if not the same as the door the contestent choose)
        if length(valid_doors) == 1 #if there is only one door left open that door 
            doors[valid_doors[1]].open = true
            return doors[valid_doors[1]]
        else #if there are 2 doors left (i.e the contestant choose the door with a car) randomly choose one of the remaining doors
            which_door = rand(valid_doors)
            doors[which_door].open = true
            return which_door
        end
    
end



open_a_door! (generic function with 1 method)


# switch_door_pick()

returns the index in {1, 2, 3} of the door that the contestant must pick if he/she switches from his/her initial door choice to the other upopened door. It is assumed in this function that one of the Doors passed has been opened by the game host and has a goat behind it. 

In [7]:

function switch_door_pick(doors::Array{Door}, first_door_id_pick::Int)
    #this function gets called if the contestent chooses to switch doors 
    switch_door = Array(1:number_doors)
    deleteat!(switch_door, [d.open for d in doors]) #takes the door that the host opened out of the options 
    deleteat!(switch_door, switch_door .== first_door_id_pick) #takes the origionally chosen door out of the options 
    return switch_door #this is the door the contestant can switch to 
end



switch_door_pick (generic function with 1 method)

# contestant_wins_car(doors::array{Door}, door_id_pick::Int)

returnns true if the contestant picked the door with the car behind it and false otherwise 

In [8]:
function contestant_wins_car(doors::Array{Door}, door_id_pick)
    #this function states wether the contestant one the car when the final discion is made 
    if doors[door_id_pick].contains == :car 
        return true 
    else
        return false  
    end 
end 



contestant_wins_car (generic function with 1 method)

# simulate_Monte_Hall(switch::Bool; verbose::Bool=true)

Uses all above functions to simulate the Monte Hall game.
    #returns true if the contestant wins and false otherwise 
    #switch argument specifies whether the contestant uses the strategy to switch doors.
    #verbose arguement specifies whether the function (for debugging and illustration) print off details of the outcome
        #what is behind each door 
        #what door the contestant picked 
        #what door the game show host opened to reveal goat
        #(if applicable) the choice of the door after the contestant switches 
        #if contestant won or lost

In [9]:
println(doors)

Door[Door(false, :car), Door(false, :goat), Door(false, :goat)]


In [10]:
#this will help when printing out the results of the simulation win=true lose=false 
do_they_win = Dict{Bool, String}(true => "The Contestant Wins!", false => "The Contestant Loses") 


function simulate_Monte_Hall(switch::Bool, verbose::Bool=true)
    #this function performs the simulation by calling all of the functions 
    doors = set_up_doors()
    first_door_id_pick = pick_a_door_at_random()
    host_pick = open_a_door!(doors, first_door_id_pick)
    
    if switch 
        #if the simulation is running with the contestant switching doors 
        switch_pick = switch_door_pick(doors, first_door_id_pick)
        outcome = contestant_wins_car(doors, switch_pick)
    else 
        #if the simulation is running with the contestant sticking with the original door 
        outcome = contestant_wins_car(doors, first_door_id_pick)
    
    end

    
    if verbose 
        
        #this is for printing out all of the results 
        for (i, d) in enumerate(doors)
                #goes through each door and pulls out the number and what is behind the door 
                println("Behind door #", i, " is a ", d.contains)
        end
        println("Contestant picked Door #", first_door_id_pick) 
        println("The game show host picked Door #", host_pick)
        switch ? 
            #this will only run if the contestant switched doors 
            println("Contestant switched to Door #", switch_pick[1]) : print("Contestant keeps Door #", first_door_id_pick )
        println(do_they_win[outcome])
    end
   
    return outcome 
end

println(doors)

Door[Door(false, :car), Door(false, :goat), Door(false, :goat)]


# Simulation 

Run 1000 simulations of the Monte Hall problem under the two stategies. Print off how many times the contestant won under both of the strategies. 

In [11]:
#running the simulation! 

#which way will the simulation be ran? 
strategies = Dict{Bool, String}(
    true => "Switch Doors",
    false => "Sticking with the original door")

#tracks the number of wins for each simulation 
wins = Dict{Bool, Int}(
    true => 0, 
    false => 0)

number_switches = length(strategies) #tracks how many switches happen. Should be the same as number of simulation 
    for s in 1:num_simulation #runs for however many simulations are ran 
        for switch in keys(wins) 
            #switch will be true(switch doors) then false(keep original door)
            wins[switch] += simulate_Monte_Hall(switch, false) #dont want to print all 1000 outcomes so verbose = false
        end
    end

print("In this simulation there were ")
for (n, switch) in enumerate(keys(wins)) 
    #will pull all of the information from the wins dictionary i = true or false, switch = # of wins for each stategy 
    num_wins = wins[switch] #number wins for each strategy 
    percent = 100 * Float64(num_wins)/Float64(num_simulation) #percentage of wins for each strategy 
    strategy = strategies[switch] #gives the string that says what stratedy was used 
    print(num_wins, " wins ", "(", percent, "%) from ", strategy)
    (n < number_switches) ? print("and") : print(".")
    
end
    

ErrorException: type Array has no field contains